In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder
import pickle

In [5]:
#the below csv states whether the employee is present in the bank or esited from the bank based on different featue here we have aroud(11 dependant and 1 independant salary feature)

data = pd.read_csv('/Users/anuhyasamudrala/Documents/Anu_uncc/Deeplearning/ANNClassification/Churn_Modelling.csv')
data.head()



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#preprcess data 
#drop irrelavant feature
data = data.drop(['RowNumber' , 'CustomerId' , 'Surname'] , axis = 1)  # might be not that used for training the model
data

In [ ]:
#convert category to numerical (onehotencoding)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data



In [ ]:
# for geography we cant make o/1/2 cant assign directly like this we have to make that to multiple rows 
#for that we will use onehotencoding 
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder.toarray()

#now we have to change the matrix to multiple coulmns using get 
onehot_encoder_geo.get_feature_names_out(['Geography'])   #column names 


In [ ]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray() , columns= onehot_encoder_geo.get_feature_names_out(['Geography']) )
geo_encoded_df

In [ ]:
#combine hot encoded geo  to our original data 

data = pd.concat([data.drop('Geography' , axis = 1) ,geo_encoded_df] , axis = 1)
data

In [59]:
#save teh above encoders in pickle file 

with open('label_encoder_gender.pkl' ,'wb') as file:
    pickle.dump(label_encoder_gender , file)
    
with open('onehot_encoder_geo.pkl' ,'wb') as file:
    pickle.dump(onehot_encoder_geo , file)
    

In [ ]:
##divide data into independanat and independant fetures
data.head()  #except exited remainign are dependant (which is o/p exited)


In [ ]:
#independet featire to fetch o/p
X = data.drop('Exited' , axis = 1)
y = data['Exited']

#train and text split 
X_train , X_test ,y_train , y_test = train_test_split(X,y , test_size= 0.2 , random_state= 42)

#scalefeature  ---normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train , X_test

In [62]:
with open('scaler.pkl' ,'wb') as file:
    pickle.dump( scaler ,file)

#### Ann classification

In [63]:
#ann
#initialise sequential 
#dense(hidden layer how many nodes) --
#activation function --sigmoid/tanh/relu/leaky relu (preffered all hideen relu ,o/p-sigmoid/relu/linear)
#optimizer --reponsible for weights 
#loss function
#metrics (classification - accuracy ) (regression - mean square error , mean ansolute error)
#store training information (logs) in folder(tensorbord)--help for visualization 

In [64]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [65]:
#building our model
model = Sequential([
    Dense(64, activation = 'relu' , input_shape = (X_train.shape[1] , )),  ##first hidden layer connect with i/p
    Dense(64, activation = 'relu'),  #hidden layer
    Dense(1 , activation  = 'sigmoid') #o/p layer
]   
)

In [ ]:
model.summary()
#params is combination of forward and bias

In [ ]:
#optimizer  and loss
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [68]:
#compile
model.compile(optimizer= opt  , loss=loss , metrics = ['accuracy'] )

In [69]:
#tensorbord
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir)

In [72]:
#set up early stopping 
early_stopping_callback = EarlyStopping(monitor= 'val_loss' , patience= 10 , restore_best_weights= True)


In [ ]:
#Train model 
history = model.fit(
    X_train, y_train , validation_data =(X_test , y_test) , epochs = 100,
    callbacks = [tensorflow_callback , early_stopping_callback ]
        
)

In [ ]:
model.save('model.h5')

In [3]:

%tensorboard --logdir logs/fit20250108-151642/

UsageError: Line magic function `%tensorboard` not found.


In [79]:
rm -rf ~/.tensorboard-info

In [ ]:
# Input fields
credit_score = st.number_input(
    'Credit Score', 
    min_value=0, 
    max_value=1000, 
    value=619, 
    help="A numerical score that represents the creditworthiness of the customer. "
         "Higher scores indicate better creditworthiness."
)

geography = st.selectbox(
    'Geography', 
    onehot_encoder_geo.categories_[0], 
    help="The region or country where the customer resides."
)

gender = st.selectbox(
    'Gender', 
    label_encoder_gender.classes_, 
    help="The gender of the customer. Please select Male or Female."
)

age = st.slider(
    'Age', 
    18, 
    100, 
    help="The age of the customer in years."
)

tenure = st.slider(
    'Tenure', 
    0, 
    50, 
    help="The number of years the customer has been with the bank."
)

num_of_products = st.number_input(
    'Number of Products Used',
    min_value=1,
    max_value=4,
    value=1,
    help="The number of products or services the customer is using with the bank. "
         "For example: a savings account, a credit card, a loan, or an investment account."
)

is_active_member = st.selectbox(
    'Is Active Member', 
    [0, 1], 
    format_func=lambda x: "Yes" if x == 1 else "No",
    help="Indicates whether the customer is currently an active member of the bank (Yes = 1, No = 0)."
)

balance = st.number_input(
    'Balance', 
    min_value=0.0, 
    value=0.0, 
    help="The total balance in the customer's bank account (in dollars)."
)

has_cr_card = st.selectbox(
    'Has Credit Card', 
    [0, 1], 
    format_func=lambda x: "Yes" if x == 1 else "No",
    help="Indicates whether the customer owns a credit card (Yes = 1, No = 0)."
)

estimated_salary = st.number_input(
    'Estimated Salary', 
    min_value=0.0, 
    help="The annual estimated salary of the customer (in dollars)."
)